# **PLOTLY GRÁFICAS**

# *Importacion y utilizacion de las librerias para compilar el codigo*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica
import plotly.express as px

# *Data import*

In [2]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
# Imprimir la ruta relativa
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

### *Creación de los df por cada archivo json*

In [3]:
# Crear un diccionario para almacenar los DataFrames
dataframes = {}
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres para obtener solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Imprimir la lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
print(nombres_df)

Lista de DataFrames creados:
['df_alfa_q_feb_2023_pachuca', 'df_alfa_q_jul_2023_pachuca', 'df_alfa_q_jun_2023_pachuca', 'df_alfa_q_mar_2023_pachuca', 'df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca', 'df_alfa_q_puebla', 'df_alfa_q_sep_2023_pachuca', 'df_alfa_q_tulancingo', 'df_enero_2024_querertaro', 'df_feb_2024_puebla', 'df_feb_2024_queretaro', 'df_financiamientos_2019_puebla', 'df_financiamientos_2019_queretaro', 'df_financiamientos_2019_tulancingo', 'df_financiamientos_2020_puebla', 'df_financiamientos_2020_queretaro', 'df_financiamientos_2020_tulancingo', 'df_financiamientos_2021_puebla', 'df_financiamientos_2021_queretaro', 'df_financiamientos_2021_tulancingo', 'df_financiamientos_2022_pachuca', 'df_financiamientos_2022_puebla', 'df_financiamientos_2022_queretaro', 'df_financiamientos_2022_tulancingo', 'df_financiamientos_2023_pachuca', 'df_financiamientos_2023_puebla', 'df_financiamientos_2023_queretaro', 'df_financiamientos_2023_tulancingo', 'df_grupos_edad_pachuca', 

________
# ***Piramide Poblacional***
# CONSULTAS
### Creación de tablas a usar

- datos_grupos_edad_pachuca

In [4]:
datos_grupos_edad_pachuca = df_grupos_edad_tulancingo[['id', 'rango', 'total', 'hombres', 'mujeres']]

In [5]:
def asignar_rango_amplio(id):
    rangos = {range(1, 5): '0-19', range(5, 8): '20-34', range(8, 11): '35-49', range(11, 14): '50-64'} 
    for rango, etiqueta in rangos.items():
        if id in rango:
            return etiqueta
    return '65+' if id > 19 else '65+'

# Asumiendo que df_grupos_edad_pachuca ya tiene la columna 'id'
datos_grupos_edad_pachuca['rango_amplio'] = datos_grupos_edad_pachuca['id'].apply(asignar_rango_amplio)

# Agrupar y sumar en una línea
datos_agrupados = datos_grupos_edad_pachuca.groupby('rango_amplio').sum().reset_index()

def create_population_pyramids(df, title, male_col, female_col, age_group_col):
    age_groups = df[age_group_col].tolist()
    men_values = df[male_col].tolist()
    women_values = df[female_col].tolist()

    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=age_groups,
        x=[-value for value in men_values],
        # text=male_data['Population'],  # Etiquetas de cantidad positiva
        name='Hombres',
        orientation='h',
        marker=dict(color='#9500ff'),
        textposition='outside',  # Posición del texto fuera de la barra
    ))

    fig.add_trace(go.Bar(
        y=age_groups,
        x=women_values,  # Coordenadas x invertidas
        # text=female_data['Population'],  # Etiquetas de cantidad positiva
        name='Mujeres',
        orientation='h',
        marker=dict(color='#ff0059'),
        textposition='outside',  # Posición del texto fuera de la barra    
    ))
    
    # Configurar el fondo de la gráfica como transparente
    # fig.update_layout({
    #     'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    #     'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    # })

    # Personalización adicional del gráfico
    fig.update_layout(
        title=title,
        barmode='overlay',
        bargap=0.35,
        xaxis=dict(title='Porcentaje'),
        yaxis=dict(
            title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
        
    )

    return fig

# Crear y mostrar la pirámide poblacional
create_population_pyramids(datos_agrupados, '', 'hombres', 'mujeres', 'rango_amplio')


In [6]:
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')
    
fig_piramide = create_population_pyramids(datos_agrupados, 'Tulancingo', 'hombres', 'mujeres', 'rango_amplio')

# Guardar la figura como archivo HTML
guardar_grafico_como_html(fig_piramide, 'g_bar_piramid_poblacion_edad_gen_tulancingo')
